# [0] [Total3D pickles] layout_emitter + 3D lighting

In [270]:
# borrowed from Total3D repo -> notebooks/parse_OR_labels-lighting-reindexed%2Bassigned-V4.ipynb
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train')

import os
import json

%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib widget

# dataset = 'sunrgbd'
dataset = 'OR'

mode = 'val'
# mode = 'train'
# mode = 'train-tmp'

# split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': 'utils_OR/openrooms/preprocessed-full'}
split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': '/home/ruizhu/Documents/Projects/semanticInverse/train/data/openrooms/list_OR_V4full/list'}

if dataset == 'sunrgbd':
    split_file = os.path.join(split_path_dict[dataset], mode + '.json')
    with open(split_file) as file:
        split = json.load(file)
elif dataset == 'OR':
    split_file = os.path.join(split_path_dict[dataset], mode + '.txt')
    with open(split_file) as f:
        mylist = f.read().splitlines() 
    split = [x.split(' ') for x in mylist]

print(len(split))

import torch
import numpy as np
from pathlib import Path
from utils.utils_total3D.utils_OR_visualize import format_layout, Box
from utils.utils_total3D.data_config import OR4XCLASSES_dict, NYU40CLASSES
from utils.utils_total3D.utils_others import OR4X_mapping_catInt_to_RGB, OR4X_mapping_catStr_to_RGB
from utils.utils_total3D.utils_OR_vis_labels import read_vis_scene_3d, RGB_to_01
import pickle
from icecream import ic


OR = 'OR45'
classes = OR4XCLASSES_dict[OR] if dataset == 'OR' else NYU40CLASSES
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catInt_to_RGB['light']), 'rb') as f:
    OR_mapping_catInt_to_RGB = pickle.load(f)[OR]
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catStr_to_RGB['light']), 'rb') as f:
    OR_mapping_catStr_to_RGB = pickle.load(f)[OR]

import random
# image_id = random.randint(0, len(split))
# image_id = 3
# pickle_path_ori = split[image_id]

# pickle_id = random.randint(0, len(split))
# pickle_id = 10
# pickle_id = 26
# pickle_id = 27

# pickle_id = 16 # blue room, sofa, window
# wall_idx, cell_i, cell_j = 2, 3, 3

# pickle_id = 6
# wall_idx, cell_i, cell_j = 4, 2, 4

pickle_id = 18 # classroom
wall_idx, cell_i, cell_j = 4, 3, 3

# random
pickle_id = 1 # [!!! 0-based]

5664


In [271]:
naming = split[pickle_id][2]
print(naming)

# naming = 'main_xml/scene0096_00/im_6.hdr'
# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'mainDiffLight_xml1/scene0017_02/im_4.hdr'
# naming = 'mainDiffMat_xml1/scene0695_03/im_26.hdr'
# pickle_path_ori = '/data/ruizhu/OR-V3-OR45_total3D_train_test_data/%06d.pkl'%pickle_id
meta_split = naming.split('/')[0]
scene_name = naming.split('/')[1]
withinsequence_id = int(naming.split('/')[2].split('.')[0].split('_')[1])
# assert pickle_id==withinsequence_id

train_test_data_path = '/data/ruizhu/OR-V4full-withMesh20210510-assign2NotReindex-OR45_total3D_train_test_data'

pickle_path_ori = '%s/%s/%s/%s'%(train_test_data_path, naming.split('/')[0], naming.split('/')[1], naming.split('/')[2].replace('im', 'layout_obj').replace('hdr', 'pkl'))


grid_size = 8

pickle_path = pickle_path_ori.replace('.pkl', '_reindexed.pkl')
pickle_emitters_path = pickle_path_ori.replace('.pkl', '_emitters.pkl')
pickle_emitters_assign_info_list_path = pickle_path_ori.replace('.pkl', '_emitters_assign_info_%dX%d_V4.pkl'%(grid_size, grid_size))

print(pickle_path)

# ======= read saved results from evaluation

# GT envmap input + overfit to val200
# results_folder = '20210503-170713-train_POD_emitterEst_lightAccuV3_bs2x4_GTsampledEnvMap-GTBRDFLight_overFitVal_fixedDataScaleCellOrder0502data'; tid=10000

# est
results_folder = '20210517-015920--tmp'; tid=0

layout_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_layout_info_tid%d-%d.pickle'%(results_folder, tid, pickle_id)
results_cells_vis_info_list_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_cells_vis_info_list_tid%d-%d.pickle'%(results_folder, tid, pickle_id)
results_emitter_input_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_input_%d.pickle'%(results_folder, pickle_id)
results_emitter_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_%d.pickle'%(results_folder, pickle_id)

with open(layout_results_pickle_path, 'rb') as f:
    layout_pickle_dict = pickle.load(f)
    print(layout_pickle_dict.keys())
with open(results_cells_vis_info_list_pickle_path, 'rb') as f:
    results_cells_vis_info_list_pickle_dict = pickle.load(f)
    print(results_cells_vis_info_list_pickle_dict.keys())
with open(results_emitter_input_pickle_path, 'rb') as f:
    emitter_input_pickle_dict = pickle.load(f)
    print(emitter_input_pickle_dict.keys())
with open(results_emitter_pickle_path, 'rb') as f:
    results_emitter_dict = pickle.load(f)
    print(results_emitter_dict.keys())

cells_vis_info_list = results_cells_vis_info_list_pickle_dict['cells_vis_info_list']
emitter_cls_prob_PRED = results_cells_vis_info_list_pickle_dict['emitter_cls_prob_PRED']
cell_info_grid_PRED = results_cells_vis_info_list_pickle_dict['cell_info_grid_PRED']

cam_R_pred = layout_pickle_dict['pre_cam_R']
pred_layout = layout_pickle_dict['pre_layout_full']
pre_boxes = layout_pickle_dict['pre_boxes']
pre_meshes = layout_pickle_dict['pre_meshes']


with open(results_emitter_input_pickle_path, 'rb') as f:
    results_emitter_input_pickle_dict = pickle.load(f)

env_scale, hdr_scale, envmap_path = results_emitter_input_pickle_dict['env_scale'], results_emitter_input_pickle_dict['hdr_scale'], results_emitter_input_pickle_dict['envmap_path']
print(env_scale, hdr_scale, envmap_path)

import pickle
with open(pickle_path, 'rb') as f:
    sequence = pickle.load(f)
print(sequence.keys())
import pickle5 as pickle
with open(pickle_path_ori, 'rb') as f:
    sequence_ori = pickle.load(f)
with open(pickle_emitters_path, 'rb') as f:
    sequence_emitters = pickle.load(f)
print(sequence_emitters.keys(), sequence_emitters['sequence_name'], sequence_emitters['withinsequence_id'])
with open(pickle_emitters_assign_info_list_path, 'rb') as f:
    sequence_emitter2wall_assign_info_dict = pickle.load(f)
print(sequence_emitter2wall_assign_info_dict.keys())

import pickle5 as pickle
emitter_representation_type = '0ambient'
emitters_prop_dict_representation_dict_path = pickle_path.replace('layout_obj', ('emitters_prop_dict_%s'%emitter_representation_type)).replace('_reindexed', '')
with open(emitters_prop_dict_representation_dict_path, 'rb') as f:
    emitters_prop_dict_representation_dict = pickle.load(f)

# transform_to_total3d_coords_dict = {'transform_R': frame.transform_R, 'transform_t': frame.transform_t}
transform_to_total3d_coords_dict_path = Path(train_test_data_path) / meta_split / scene_name / ('transform_to_total3d_coords_dict_%d.pkl'%withinsequence_id)
import pickle5 as pickle
with open(transform_to_total3d_coords_dict_path, 'rb') as f:
    transform_to_total3d_coords_dict = pickle.load(f)
transform_R, transform_t = transform_to_total3d_coords_dict['transform_R'], transform_to_total3d_coords_dict['transform_t']

# RAW_data_path = '/newfoundland2/ruizhu/siggraphasia20dataset/layout_labels_V4full-detachEmitterRERERE20210420/'
# emitter_dict_pickle_file = Path(RAW_data_path) / meta_split / scene_name / ('emitter_dict_%s_%d.pickle'%('0ambient', withinsequence_id))
# with open(emitter_dict_pickle_file, "rb") as f:
#     emitter_dict_RAW = pickle.load(f)


%matplotlib widget

gt_obj_paths_list = [x['obj_path'] for x in sequence['boxes']['bdb2D_full']]
gt_boxes_dict_RE = {key: [x[key] for x in sequence['boxes']['bdb3D_full']] for key in sequence['boxes']['bdb3D_full'][0]}
len_keys = [len(gt_boxes_dict_RE[key]) for key in gt_boxes_dict_RE]
assert len(list(set(len_keys))) == 1
gt_obj_paths_list = [x['obj_path'] for x in sequence['boxes']['bdb2D_full']]
random_id_list = [x['random_id'] for x in sequence['boxes']['bdb2D_full']]

gt_obj_paths_list_normalized = [x.replace('/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped', '/home/ruizhu/Documents/data/OR-sampledMeshes').replace('alignedNew.obj', 'alignedNew_normalized.obj') for x in gt_obj_paths_list]
assert len(gt_obj_paths_list_normalized)==len(gt_obj_paths_list)==len_keys[0]
gt_boxes_dict_RE['obj_path_normalized'] = gt_obj_paths_list_normalized
gt_boxes_dict_RE['obj_path_original'] = gt_obj_paths_list
gt_boxes_dict_RE['random_id'] = random_id_list
gt_boxes_dict_RE['bdb2d'] = [[x['x1'], x['y1'], x['x2'], x['y2']] for x in sequence['boxes']['bdb2D_full']]
gt_boxes_dict_RE['obj_masks'] = [x for x in sequence['boxes']['mask']]
assert len(gt_boxes_dict_RE['obj_masks']) == len(gt_boxes_dict_RE['bdb2d'])
assert len(gt_boxes_dict_RE['obj_masks']) == len(sequence['boxes']['if_valid'])
for bbox_idx in range(len(gt_boxes_dict_RE['if_valid'])):
    if gt_boxes_dict_RE['obj_masks'][bbox_idx] is None:
        gt_boxes_dict_RE['if_valid'][bbox_idx] = False
#     if sequence['boxes']['bdb2D_full']['if_is_indoor_obj'] == False:
#         gt_boxes_dict_RE['if_valid'][bbox_idx] = False
#     else:

# gt_layout = sequence['layout']['bdb3D']
# gt_layout = sequence_ori['layout']['bdb3D']
gt_layout = sequence_ori['layout']

from utils.utils_total3D.data_config import Dataset_Config
from utils.utils_total3D.utils_OR_layout import to_dict_tensor

from utils.utils_total3D.utils_OR_cam import get_rotation_matrix_gt

OR = 'OR45'
dataset_config = Dataset_Config(dataset, OR=OR, version='V3', paths={'total3D_lists_path': '../train/data/openrooms/list_OR_V4full', 'layout_emitter_path': '/data/ruizhu/OR-V4full-OR45_total3D_train_test_data'})
bins_tensor = to_dict_tensor(dataset_config.bins, if_cuda=False)
cam_R_gt = get_rotation_matrix_gt(bins_tensor,
                                  torch.zeros([1]).long()+sequence['camera']['pitch_cls'], torch.zeros([1])+sequence['camera']['pitch_reg'],
                                  torch.zeros([1]).long()+sequence['camera']['roll_cls'], torch.zeros([1])+sequence['camera']['roll_reg']).squeeze().numpy()

# ======== emitters_list
emitters_obj_list = []
for x in range(sequence_emitters['boxes']['bdb3D'].shape[0]):
    obj_dict_new = {'obj_box_3d': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'random_id': sequence_emitters['boxes']['random_id'][x], \
                    'emitter_prop': sequence_emitters['boxes']['emitter_prop'][x], \
                    'bdb3D_emitter_part': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'cat_id': sequence_emitters['boxes']['size_cls'][x], \
                    'cat_name': classes[sequence_emitters['boxes']['size_cls'][x]], 'cat_color': RGB_to_01(OR_mapping_catInt_to_RGB[sequence_emitters['boxes']['size_cls'][x]])}
    obj_random_id = sequence_emitters['boxes']['random_id'][x]
    emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']
    if sequence_emitters['boxes']['emitter_prop'][x]['obj_type'] == 'window' and 'light_center_world_total3d' in emitter_prop_total3d:
        light_center_world_total3d = emitter_prop_total3d['light_center_world_total3d'].reshape(3, 1)
        light_axis_world_total3d = emitter_prop_total3d['light_axis_world_total3d'].reshape(3, 1)
    else:
        light_center_world_total3d = np.zeros((3, 1), dtype=np.float32)
        light_axis_world_total3d = np.zeros((3, 1), dtype=np.float32)
    obj_dict_new['light_world_total3d_centeraxis'] = [light_center_world_total3d, light_axis_world_total3d]

    obj_dict_new['emitter_prop'].update({'emitter_rgb_float': emitter_prop_total3d['intensity']})

    emitters_obj_list.append(obj_dict_new)

emitter2wall_assign_info_list = sequence_emitter2wall_assign_info_dict['emitter2wall_assign_info_list']
cell_info_grid_GT_includeempty = sequence_emitter2wall_assign_info_dict['cell_info_grid']
cell_info_grid_GT = []
for wall_idxx in range(6):
    for i in range(grid_size):
        for j in range(grid_size):
            cell_info = cell_info_grid_GT_includeempty[wall_idxx * grid_size**2 + i * grid_size + j]
            if cell_info['obj_type'] is None:
                continue
            cell_info['wallidx_i_j'] = (wall_idxx, i, j)
            cell_info_grid_GT.append(cell_info)

            obj_random_id = cell_info['emitter_info']['random_id']
            emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']


#             cell_intensity = np.array([emitter_prop_total3d['intensity_scale255'] * x * 255.for x in emitter_prop_total3d['intensity_scaled01']]) # intensity_scaled: [0., 1.]
            env_scale_multiplier = env_scale if cell_info['obj_type'] == 'window' else 1.
            cell_intensity = np.array(emitter_prop_total3d['intensity']) * env_scale_multiplier * hdr_scale
            cell_info['emitter_info']['intensity_scalelog'] = np.log(np.clip(np.linalg.norm(cell_intensity.flatten()) + 1., 1., np.inf))
            cell_info['emitter_info']['intensity'] = [x * env_scale_multiplier * hdr_scale for x in emitter_prop_total3d['intensity']]
#             ic(emitter_prop_total3d['intensity'], cell_info['emitter_info']['intensity'], env_scale, hdr_scale)
            intensity_scale255 = max(cell_info['emitter_info']['intensity']) / 255.
            intensity_scaled01 = [np.clip(x / (intensity_scale255+1e-5) / 255., 0., 1.) for x in cell_info['emitter_info']['intensity']]
            cell_info['emitter_info']['intensity_scaled01'] = intensity_scaled01
            

            # other representation-specific params
            if cell_info['obj_type'] == 'window':
                cell_info['emitter_info']['lamb'] = emitter_prop_total3d['lamb']
                cell_info['emitter_info']['light_dir_abs'] = emitter_prop_total3d['light_axis_world_total3d'].reshape(3,)
#                 print(cell_info['emitter_info']['light_dir_abs'], obj_random_id)
                if emitter_representation_type in ['1ambient']:
                    cell_info['emitter_info']['ambient'] = emitter_prop_total3d['ambient']
                if emitter_representation_type in ['2ambient']:
                    cell_info['emitter_info']['ambientL'] = emitter_prop_total3d['ambientL']
                    cell_info['emitter_info']['ambientR'] = emitter_prop_total3d['ambientR']
                    
#             print(wall_idxx, i,j, cell_info['emitter_info']['normal_outside'])



plt.figure(figsize=(15, 8))
ax_2d = plt.gca()

grid_prob = sequence_emitter2wall_assign_info_dict['wall_grid_prob'].reshape(6, -1)
import imageio
rgb_img = imageio.imread(sequence['rgb_img_path'])
gt_meshes = gt_obj_paths_list

scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, cam_R_pred, gt_layout, pred_layout, gt_boxes_dict_RE, pre_boxes, gt_meshes, pre_meshes, dataset=dataset, OR='OR45', \
                emitters_obj_list_gt=emitters_obj_list, emitter2wall_assign_info_list_gt=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=emitter_cls_prob_PRED, emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, cell_info_grid_PRED=cell_info_grid_PRED, \
                grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl', 'root_path': '/home/ruizhu/Documents/Projects/semanticInverse'}, \
                pickle_id=pickle_id, if_hide_invalid_cats=True, \
                if_use_vtk=True, if_index_faces_with_basis=True, \
                summary_path='', transform_R=transform_R, transform_t=transform_t)

%matplotlib widget

img_map2 = scene_box.draw_projected_bdb3d('both', return_plt=True, if_save=False, save_path='', if_use_plt=True, if_vis_2dbbox=False, 
                                          if_vis_2dbbox_iou=False, if_vis_3dbboxproj=True, if_vis_2dbbox_random_id=False)
# img_map2.show()

fig_3d, _, ax_3ds_scene = scene_box.draw_3D_scene_plt('both', if_show_objs=True, hide_random_id=False, if_debug=True, hide_cells=False, if_dump_to_mesh=False)
#                                                    points_backproj=points_backproj_total3D, points_backproj_color=img_resize)
# fig_3d, return_dict, ax_3ds_scene = scene_box.draw_3D_scene_plt('GT', if_debug=True, if_show_objs=True, hide_random_id=False, hide_cells=False, if_dump_to_mesh=True, if_show_emitter=True, fig_scale=1., \
#                                                    if_show_cell_normals=True, if_show_cell_meshgrid=True)

mainDiffMat_xml/scene0403_01/im_12.hdr
/data/ruizhu/OR-V4full-withMesh20210510-assign2NotReindex-OR45_total3D_train_test_data/mainDiffMat_xml/scene0403_01/layout_obj_12_reindexed.pkl
dict_keys(['rgb_img_path', 'bins_tensor', 'est_data', 'gt_cam_R', 'cam_K', 'gt_layout', 'pre_layout', 'gt_layout_full', 'pre_layout_full', 'gt_meshes', 'pre_meshes', 'gt_boxes', 'pre_boxes', 'pre_cam_R', 'image'])
dict_keys(['cells_vis_info_list', 'cell_info_grid_GT', 'cell_info_grid_PRED', 'emitter_cls_prob_PRED', 'emitter_cls_prob_GT', 'pre_layout', 'pre_cam_R', 'envmap_lightAccu_mean_vis_GT'])
dict_keys(['hdr_scale', 'normalPred_lightAccu', 'depthPred_lightAccu', 'envmapsPredImage_lightAccu', 'env_scale', 'envmap_path'])
dict_keys(['envmap_lightAccu_mean', 'points_sampled_mask_expanded', 'cell_normal_outside_label', 'emitter_cell_axis_abs_est', 'emitter_cell_axis_abs_gt', 'window_mask', 'envmap_lightAccu', 'scattered_light', 'emitter_outdirs_meshgrid_Total3D_outside', 'normal_outside_Total3D', 'depthPre

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 [Skipped] 41:door [invalid class of dataset OR]
 [Skipped] 41:door [invalid class of dataset OR]
[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 [Skipped] 41:door [invalid class of dataset OR]
 [Skipped] 41:door [invalid class of dataset OR]


In [272]:
T_LightNet2Total3D_rightmult = results_emitter_dict['T_LightNet2Total3D_rightmult'].squeeze() # [3, 3]

# transform_R, transform_t = transform_to_total3d_coords_dict['transform_R'], transform_to_total3d_coords_dict['transform_t']

points_backproj = results_emitter_dict['points_backproj'].transpose(1, 2, 0)
# points_backproj = transform_R @ (points_backproj.reshape(-1, 3).T) + transform_t
points_backproj_total3D = points_backproj.reshape(-1, 3) @ T_LightNet2Total3D_rightmult
points_backproj_total3D = points_backproj_total3D.reshape(240, 320, 3)

import cv2
img_resize = cv2.resize(rgb_img, (320, 240), interpolation = cv2.INTER_AREA )

# get object insertion data

In [283]:
material_dict = {'GT': {'albedo': results_emitter_dict['albedoGT'].transpose(1, 2, 0), \
                        'rough': results_emitter_dict['roughGT'].transpose(1, 2, 0), \
                         'semseg': results_emitter_dict['semseg_label']}, \
                'prediction': {'albedo': results_emitter_dict['albedoPred'].transpose(1, 2, 0), \
                        'rough': results_emitter_dict['roughPred'].transpose(1, 2, 0), \
                         'semseg': results_emitter_dict['semseg_label']}
                }

from pathlib import Path
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding/utils_OR')
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding')
from utils_OR.utils_OR_xml import get_XML_root, parse_XML_for_shapes
import random
from utils_OR.utils_OR_xml import gen_random_str, transformToXml
from copy import deepcopy
import xml.etree.ElementTree as et

random_str = gen_random_str(5)
tmp_folder = Path('/home/ruizhu/Downloads/tmp/%s'%random_str)
tmp_folder.mkdir()

dataset_main_path = Path('/newfoundland2/ruizhu/siggraphasia20dataset')
envmap_root = '/home/ruizhu/Documents/data/EnvDataset/'



scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, cam_R_pred, gt_layout, pred_layout, gt_boxes_dict_RE, pre_boxes, gt_meshes, pre_meshes, dataset=dataset, OR='OR45', \
                emitters_obj_list_gt=emitters_obj_list, emitter2wall_assign_info_list_gt=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=emitter_cls_prob_PRED, emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, cell_info_grid_PRED=cell_info_grid_PRED, \
                grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl', 'root_path': '/home/ruizhu/Documents/Projects/semanticInverse'}, \
                pickle_id=pickle_id, if_hide_invalid_cats=True, \
                if_use_vtk=True, if_index_faces_with_basis=True, \
                summary_path=str(tmp_folder), transform_R=transform_R, transform_t=transform_t, hdr_scale=hdr_scale, env_scale=env_scale, material_dict=material_dict)

split_type = 'GT'
# split_type = 'prediction'

target_xml_file = scene_box.scene_to_xml(split_type=split_type)

# ----- switch
if split_type=='GT':
    depth_use = results_emitter_dict['depthGT'].squeeze()
    normal_use = results_emitter_dict['normalGT'].squeeze()
    cam_R_use = cam_R_gt # [axes hstacked]

elif split_type=='prediction':
    depth_use = results_emitter_dict['depthPred'].squeeze()
    normal_use = results_emitter_dict['normalPred'].squeeze()
    cam_R_use = cam_R_pred


# --- envmaps
if 'im_envmap_gen' in scene_box.sg_envmap_info_dict[split_type]:
    im_envmap_gen = scene_box.sg_envmap_info_dict[split_type]['im_envmap_gen']
    im_envmap_gen_path = tmp_folder / 'im_envmap_gen.hdr'
    cv2.imwrite(str(im_envmap_gen_path), im_envmap_gen[:, :, ::-1])

    im_envmap_ori = loadHdr_simple(envmap_path)
    im_envmap_ori_uint8, im_envmap_ori_scale = to_nonhdr(im_envmap_ori)
    im_envmap_gen_uint8, _ = to_nonhdr(im_envmap_gen, scale=im_envmap_ori_scale)

    %matplotlib widget
    plt.figure(figsize=(8, 3))
    ax = plt.subplot(121)
    plt.imshow(im_envmap_ori_uint8)
    ax.set_title('Ori env map (global RAW coords)')
    ax = plt.subplot(122)
    plt.imshow(im_envmap_gen_uint8)
    ax.set_title('render SG env map (global RAW coords)')

    plt.show()

[!!!!] no mask for albedo for ceiling
[!!!!] median albedo for floor [0.3488648533821106, 0.2976526618003845, 0.2508402466773987]
[!!!!] median albedo for wall [0.0824141874909401, 0.07592610269784927, 0.20109573006629944]
Written wall to /home/ruizhu/Downloads/tmp/QS46H/mesh_wall0-tid0-1.obj
0 ---> ceiling
Written wall to /home/ruizhu/Downloads/tmp/QS46H/mesh_wall1-tid0-1.obj
1 ---> floor
Written wall to /home/ruizhu/Downloads/tmp/QS46H/mesh_wall2-tid0-1.obj
2 ---> wall
Written wall to /home/ruizhu/Downloads/tmp/QS46H/mesh_wall3-tid0-1.obj
3 ---> wall
Written wall to /home/ruizhu/Downloads/tmp/QS46H/mesh_wall4-tid0-1.obj
4 ---> wall
Written wall to /home/ruizhu/Downloads/tmp/QS46H/mesh_wall5-tid0-1.obj
5 ---> wall
Written lamps to /home/ruizhu/Downloads/tmp/QS46H/mesh_lamp_cell0-tid0-1.obj
 [Skipped] 41:door [invalid class of dataset OR]
Written objs to /home/ruizhu/Downloads/tmp/QS46H/mesh_obj0-tid0-1.obj
Written objs to /home/ruizhu/Downloads/tmp/QS46H/mesh_obj1-tid0-1.obj
Written o

## gen data for insertion

In [282]:
T_LightNet2Total3D_rightmult = results_emitter_dict['T_LightNet2Total3D_rightmult'].squeeze() # [3, 3]

# points_backproj = results_emitter_dict['points_backproj'].transpose(1, 2, 0)
# points_backproj_total3D = points_backproj.reshape(-1, 3) @ T_LightNet2Total3D_rightmult
# points_backproj_RAW = transform_R.T @ ((points_backproj_total3D.T) - transform_t)
# points_backproj_RAW = (points_backproj_RAW.T).reshape(240, 320, 3)

im_width = 320
im_height = 240
f = sequence['camera']['K'][0][0] / 2.
u0 = im_width / 2.
v0 = im_height / 2.

uu, vv = np.meshgrid(range(im_width), range(im_height))
z = -depth_use

x = - (uu - u0) / f * z
y = (vv - v0) / f * z
points = np.stack([x.squeeze(), y.squeeze(), z.squeeze()])
points_backproj = points.transpose(1, 2, 0)
points_backproj_total3D = points_backproj.reshape(-1, 3) @ T_LightNet2Total3D_rightmult
# points_backproj_total3D = points_backproj_total3D.reshape(240, 320, 3)
points_backproj_RAW = transform_R.T @ ((points_backproj_total3D.T) - transform_t)
points_backproj_RAW = (points_backproj_RAW.T).reshape(240, 320, 3)

import cv2
img_resize = cv2.resize(rgb_img, (320, 240), interpolation = cv2.INTER_AREA )

pix_insert = [200, 150] # [x,y]
# pix_insert = [100, 200] # [x,y]
%matplotlib widget

fig = plt.figure(figsize=(6, 4))
plt.imshow(img_resize)
plt.scatter(pix_insert[0], pix_insert[1], s=20, color='k')
plt.show()
fig.savefig('plot.png')
insert_plot = plt.imread('plot.png')

p3d_insert = points_backproj_RAW[pix_insert[1], pix_insert[0]]
print(p3d_insert)


# --- other transforms
normal_LightNet = normal_use.transpose(1, 2, 0)
normal_LightNet_total3D = normal_LightNet.reshape(-1, 3) @ T_LightNet2Total3D_rightmult
# points_backproj_total3D = points_backproj_total3D.reshape(240, 320, 3)
normal_RAW = transform_R.T @ ((normal_LightNet_total3D.T) - transform_t)
normal_RAW = (normal_RAW.T).reshape(240, 320, 3)
print(normal_LightNet_total3D.reshape(240, 320, 3)[pix_insert[1], pix_insert[0]])

cam_axes_RAW_rows = (transform_R.T @ (cam_R_use)).T
cam_origin_RAW = (transform_R.T @ (- transform_t)).T
cam_target_RAW = cam_origin_RAW + cam_axes_RAW_rows[0]
cam_up_RAW = cam_axes_RAW_rows[1]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.2177 -0.2686  1.155 ]
[-0.0558  0.9984  0.0049]


### render ori envmap

In [284]:
main_xml_file = Path(xml_path) / sequence['sub_name'] / sequence['scene_name'] / ('%s.xml'%sequence['meta_name'])


import xml.etree.ElementTree as et
from utils_OR.utils_OR_imageops import loadHdr_simple, to_nonhdr

random_str = gen_random_str(5)
tmp_folder = Path('/home/ruizhu/Downloads/tmp/%s'%random_str)
tmp_folder.mkdir()
# rec_envmap_filename = tmp_folder / 'rec_envmap.hdr'
# cv2.imwrite(str(rec_envmap_filename), recImg_gen[:, :, ::-1].astype(np.float32))
# print('===== Written envmap to ' + str(rec_envmap_filename))

root = get_XML_root(main_xml_file)
rec_root = deepcopy(root )
emitters_env = rec_root.findall('emitter')
assert len(emitters_env) == 1
emitter_env = emitters_env[0]
assert emitter_env.get('type') == 'envmap'
# emitter_env.findall('string')[0].set('value', str(rec_envmap_filename)) # use new SG env map!!!!!
# emitter_env.findall('float')[0].set('value', '0.0') # TURN OFF env map
rec_root.findall('sensor')[0].findall('sampler')[0].findall('integer')[0].set('value', '200') # Change samples
# rec_root.findall('sensor')[0].findall('sampler')[0].set('type', 'independent')

for shape in rec_root.findall('shape'):
    if len(shape.findall('emitter'))>0:
        rgb = shape.findall('emitter')[0].findall('rgb')[0]
        rgb.set('value', ' '.join([str(float(x)*hdr_scale) for x in rgb.get('value').split(' ')]))

sensor = rec_root.findall('sensor')[0]
sensor.set('type', 'envmap')
film = sensor.findall('film')[0]
integers = film.findall('integer')
for integer in integers:
    if integer.get('name') == 'width':
        integer.set('value', '%d' % 512 )
    elif integer.get('name') == 'height':
        integer.set('value', '%d' % 256 )

# center = [0., 0., 0.]
center = p3d_insert

zAxis = [0, 1., 0]
camTransform = et.SubElement(sensor, 'transform')
camTransform.set('type', 'toWorld')
lookAt = et.SubElement(camTransform, 'lookAt')
lookAt.set('origin', '%.3f %.3f %.3f' \
        % (center[0], center[1], center[2] ) )
lookAt.set('target', '%.3f %.3f %.3f' \
        % (center[0] + zAxis[0], center[1] + zAxis[1], center[2] + zAxis[2] ) )
lookAt.set('up', '-1 0 0')

# shapes = [x for x in rec_root.findall('shape')]
# for x in shapes:
#     rec_root.remove(x)

# bsdfs = [x for x in rec_root.findall('bsdf')]
# for x in bsdfs:
#     rec_root.remove(x)

xmlString = transformToXml(rec_root )
new_xml_path = str(main_xml_file).replace('.xml', '_rerennder-GT-envmap_%s.xml'%random_str)
with open(str(new_xml_path), 'w') as xmlOut:
    xmlOut.write(xmlString )
print('===== Written envmap XML to ' + str(new_xml_path))

print('===== Rendering to ' + str(tmp_folder))
rendering_command = 'cd %s && optixRenderer -f %s -o %s -m 0 --gpuIds 0' \
    % (str(dataset_main_path), str(Path(new_xml_path).relative_to(dataset_main_path)), str(tmp_folder / 'rerender-envmap-GTxml.rgbe'),)
print(rendering_command)
os.system(rendering_command)
os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
print('===== Moved to ', str(tmp_folder / Path(new_xml_path).name))

im_rerender_GTxml = loadHdr_simple(str(tmp_folder / 'rerender-envmap-GTxml_1.rgbe'))
im_rerender_GTxml_uint8, im_rerender_GTxml_scale_orienvmap = to_nonhdr(im_rerender_GTxml)

%matplotlib widget

plt.figure(figsize=(8, 4))
plt.imshow(im_rerender_GTxml_uint8)
plt.show()

===== Written envmap XML to /newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml/scene0403_01/mainDiffMat_rerennder-GT-envmap_7JCVW.xml
===== Rendering to /home/ruizhu/Downloads/tmp/7JCVW
cd /newfoundland2/ruizhu/siggraphasia20dataset && optixRenderer -f code/Routine/scenes/xml/scene0403_01/mainDiffMat_rerennder-GT-envmap_7JCVW.xml -o /home/ruizhu/Downloads/tmp/7JCVW/rerender-envmap-GTxml.rgbe -m 0 --gpuIds 0
===== Moved to  /home/ruizhu/Downloads/tmp/7JCVW/mainDiffMat_rerennder-GT-envmap_7JCVW.xml
/home/ruizhu/Downloads/tmp/7JCVW/rerender-envmap-GTxml_1.rgbe 258.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### render our representation envmap

In [170]:
target_xml_file

PosixPath('/home/ruizhu/Downloads/tmp/XQQUR/xml-GT-tid0-21.xml')

In [285]:

root = get_XML_root(str(target_xml_file))
rec_root = deepcopy(root )

emitters_env = rec_root.findall('emitter')
assert len(emitters_env) == 1
emitter_env = emitters_env[0]
assert emitter_env.get('type') == 'envmap'
if 'im_envmap_gen' in scene_box.sg_envmap_info_dict[split_type]:
    emitter_env.findall('string')[0].set('value', str(im_envmap_gen_path))
    emitter_env.findall('float')[0].set('value', str(env_scale))
else:
#     rec_root.remove(emitter_env)
    emitter_env.findall('string')[0].set('value', "../../../../../EnvDataset/1538L.hdr")
    emitter_env.findall('float')[0].set('value', str(0.00000001))

sensor = rec_root.findall('sensor')[0]
sensor.set('type', 'envmap')
film = sensor.findall('film')[0]
integers = film.findall('integer')
for integer in integers:
    if integer.get('name') == 'width':
        integer.set('value', '%d' % 512 )
    elif integer.get('name') == 'height':
        integer.set('value', '%d' % 256 )

# center = [0., 0., 0.]
center = p3d_insert
zAxis = [0, 1., 0]
camTransform = et.SubElement(sensor, 'transform')
camTransform.set('type', 'toWorld')
lookAt = et.SubElement(camTransform, 'lookAt')
lookAt.set('origin', '%.3f %.3f %.3f' \
        % (center[0], center[1], center[2] ) )
lookAt.set('target', '%.3f %.3f %.3f' \
        % (center[0] + zAxis[0], center[1] + zAxis[1], center[2] + zAxis[2] ) )
lookAt.set('up', '-1 0 0')

rec_root.findall('sensor')[0].findall('sampler')[0].findall('integer')[0].set('value', '200') # Change samples
# rec_root.findall('sensor')[0].findall('sampler')[0].set('type', 'independent')

# new_xml_path = str(main_xml_file).replace('.xml', '_emptyRoom_%s.xml'%random_str)
new_xml_path = '/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml1/scene0552_00/main_tmp.xml'

xmlString = transformToXml(rec_root )
with open(str(new_xml_path), 'w') as xmlOut:
    xmlOut.write(xmlString )
print('===== Written envmap XML to' + str(new_xml_path))

print('===== Rendering to ' + str(tmp_folder))
rendering_path = tmp_folder / ('rerender-envmap-%s_1.rgbe'%split_type)
if rendering_path.exists():
    rendering_path.unlink()
    
rendering_command = 'cd %s && optixRenderer -f %s -o %s -m 0 --gpuIds 0' \
    % (str(dataset_main_path), str(Path(new_xml_path).relative_to(dataset_main_path)), str(tmp_folder / ('rerender-envmap-%s.rgbe'%split_type)),)
print(rendering_command)
try:
    os.system(rendering_command)
except:
    print('Rendering failed!')
os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
print('===== Moved to ', str(tmp_folder / Path(new_xml_path).name))


im_rerender = loadHdr_simple(str(rendering_path))
im_rerender_uint8, _ = to_nonhdr(im_rerender, scale=im_rerender_scale_orienvmap)

%matplotlib widget

plt.figure(figsize=(8, 4))
plt.imshow(im_rerender_uint8)
plt.show()

===== Written envmap XML to/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml1/scene0552_00/main_tmp.xml
===== Rendering to /home/ruizhu/Downloads/tmp/7JCVW
cd /newfoundland2/ruizhu/siggraphasia20dataset && optixRenderer -f code/Routine/scenes/xml1/scene0552_00/main_tmp.xml -o /home/ruizhu/Downloads/tmp/7JCVW/rerender-envmap-GT.rgbe -m 0 --gpuIds 0
===== Moved to  /home/ruizhu/Downloads/tmp/7JCVW/main_tmp.xml
/home/ruizhu/Downloads/tmp/7JCVW/rerender-envmap-GT_1.rgbe 258.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [178]:
im_rerender = loadHdr_simple(str(rendering_path))
im_rerender_uint8, _ = to_nonhdr(im_rerender, scale=1/10.)

%matplotlib widget

plt.figure(figsize=(8, 4))
plt.imshow(im_rerender_uint8)
plt.show()

/home/ruizhu/Downloads/tmp/ZP5PJ/rerender-envmap-GT_1.rgbe 107.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### write dict for object insertion

In [382]:
insertion_data_dict = {}
insertion_data_dict['im'] = img_resize # [im_height, im_width, 3]
insertion_data_dict['depth'] = depth_use # [im_height, im_width]; dpeth w.r.t. camera center; positive
insertion_data_dict['normal'] = normal_RAW # [im_height, im_width, 3]; in OptixRenderer coordinates
insertion_data_dict['cam'] = [cam_origin_RAW.flatten(), cam_target_RAW.flatten(), cam_up_RAW.flatten()] # [(3,), (3,), (3,)]; origin, target and up; following cam.txt
insertion_data_dict['insert_xy'] = pix_insert # insertion point in 2D; (u, v); u: image upper-left(0) to upper-right(im_width-1); v: image upper-left(0) to lower-left(im_height-1)
insertion_data_dict['insert_p3d'] = p3d_insert # insertion point in 3D; in OptixRenderer coordinates
insertion_data_dict['envmap'] = im_rerender # envmap in 3D; (256, 512, 3); in OptixRenderer coordinates (global envmap; same as GT global envmaps e.g. EnvDataset/81L.hdr)
insertion_data_dict['insert_point_plot'] = insert_plot

pickle_save_path = '../tmp/insertion_data_dict_%s-%s-%d-%s_%d-%d.pickle'%(meta_split, scene_name, pickle_id, split_type, pix_insert[0], pix_insert[1])
with open(str(pickle_save_path),"wb") as f:
    pickle.dump(insertion_data_dict, f)
print(pickle_save_path)



../tmp/insertion_data_dict_main_xml1-scene0552_00-3-prediction_150-210.pickle


### write xml to mesh

In [249]:
from utils_OR.utils_OR_xml import xml_scene_to_mesh

moved_path = tmp_folder / Path(new_xml_path).name

mesh_path = '../test_mesh_RECON_oneBoxLight_bunny.obj'
if Path(mesh_path).exists():
    Path(mesh_path).unlink()
insertion_path = '/home/ruizhu/Documents/Projects/Total3DUnderstanding/VirtualObjectInsertion'
xml_scene_to_mesh(str(moved_path), mesh_path, skipped_walls=False, base_path=str(insertion_path))

print('===== Layout mesh written to: %s'%(str(mesh_path)))

18it [00:00, 106.90it/s]

/home/ruizhu/Downloads/tmp/IJ7BK/mesh_wall0-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_wall1-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_wall2-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch0_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch1_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch2_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch3_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch4_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch5_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch6_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_patch7_on3-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_wall4-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_wall5-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_lamp_cell0-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_lamp_cell1-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_lamp_cell2-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_lamp_cell3-tid0-3.obj
/home

22it [00:00, 14.54it/s] 

/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj5-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj6-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj7-tid0-3.obj


25it [00:02,  4.96it/s]

/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj8-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj9-tid0-3.obj


27it [00:03,  3.03it/s]

/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj10-tid0-3.obj
/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj11-tid0-3.obj


30it [00:05,  5.93it/s]


/home/ruizhu/Downloads/tmp/IJ7BK/mesh_obj12-tid0-3.obj
Mesh saved to ../test_mesh_RECON_oneBoxLight_bunny.obj; loaded from /home/ruizhu/Downloads/tmp/IJ7BK/main_tmp.xml.
===== Layout mesh written to: ../test_mesh_RECON_oneBoxLight_bunny.obj


In [170]:
str(moved_path)

'/home/ruizhu/Downloads/tmp/SSKKN/main_tmp.xml'

# tmp

In [15]:
# fig, axes = plt.subplots(ncols=3, sharey="row")
# axes[0].plot([0,1],[0,1])
# axes[1].plot([0,1],[1,2])
# axes[2].plot([0,1],[2,3])

# fig_3d, ax_3ds_scene
with open('fig1.pkl', 'wb') as f:
    pickle.dump(fig_3d, f)  

with open('fig1.pkl', 'rb') as f:
    fig2 = pickle.load(f)
ax_master = fig2.axes[0]
for ax in fig2.axes:
    print(ax)
    if ax is not ax_master:
        ax_master.get_shared_y_axes().join(ax_master, ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Axes3DSubplot(0.125,0.164744;0.352273x0.660511)


In [30]:
im = scene_box.draw3D('GT', if_return_img=True, if_save_img=False, if_save_obj=False, save_path_without_suffix = 'recon')['im']

%matplotlib widget

plt.figure()
plt.imshow(im)
plt.show()

dict_keys(['hdr_scale', 'normalPred_lightAccu', 'depthPred_lightAccu', 'envmapsPredImage_lightAccu', 'env_scale', 'envmap_path'])

## read GT envmap

In [131]:
envmap_path

'/home/ruizhu/Documents/data/EnvDataset/81L.hdr'

In [288]:
from pathlib import Path
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding/utils_OR')
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding')
from utils_OR.utils_OR_xml import get_XML_root, parse_XML_for_shapes
import random

envmap_root = '/home/ruizhu/Documents/data/EnvDataset/'

from utils_OR.utils_OR_imageops import loadHdr_simple, to_nonhdr
im_envmap_ori = loadHdr_simple(envmap_path)
im_envmap_ori_uint8, im_envmap_ori_scale = to_nonhdr(im_envmap_ori)
print(im_envmap_ori_scale)


from utils_OR.utils_OR_imageops import loadHdr, loadImage, scaleHdr, load_OR_hdr_uint8

# lightId = int(light_path.split('/')[-1].split('.')[0].replace('light0ambient', ''))

# img_path_halfenvmap = Path(xml_path) / sequence['sub_name'] / sequence['scene_name'] / \
# ('%s-light%d_1.rgbe' % (sequence['meta_name'].replace('mainDiffMat', 'main'), lightId ))
# print(img_path_halfenvmap)
# extra_scale = 1.

# im_uint8_halfenvmap, (im_halfenvmap, _, _) = load_OR_hdr_uint8(img_path_halfenvmap, scale=im_envmap_ori_scale * extra_scale)
# recImg_gen_uint8, _ = to_nonhdr(recImg_gen, scale=im_envmap_ori_scale * extra_scale)



plt.figure(figsize=(8, 4))
# ax = plt.subplot(221)
# plt.imshow(im_uint8_halfenvmap)
# ax.set_title('ori half env map')

ax = plt.subplot(121)
plt.imshow(im_envmap_ori_uint8)
ax.set_title('Ori env map (global RAW coords)')

# ax = plt.subplot(122)
# plt.imshow(im_envmap_gen_uint8)
# ax.set_title('render SG env map (global RAW coords)')
plt.show()

/home/ruizhu/Documents/data/EnvDataset/81L.hdr 50944.0
1.0378378378378377


In [219]:
im_envmap_ori_uint8.shape

(512, 1024, 3)

### get gen SG map

In [310]:
sg_params_list = scene_box.emitter_info_dict[split_type]['sg_params_list']

sg_params = sg_params_list[0]
intensity_SG = sg_params['intensity'] / hdr_scale / env_scale
lamb_SG = sg_params['lamb']
light_dir_abs_RAW = sg_params['light_dir_abs']

hdr_scale, env_scale, sg_params

(0.27234042553191484,
 91.3555,
 {'wall_idx': 3,
  'intensity': array([30890.8085, 28302.9149, 17371.4655]),
  'lamb': array([70.], dtype=float32),
  'light_dir_abs': array([-0.565 ,  0.3309,  0.7558])})

In [334]:
from utils_OR.DatasetCreation import SGOptim
envOptim = SGOptim.SGEnvOptimSky(weightValue=np.zeros(3,), thetaValue=0., phiValue=0., ambientValue=np.zeros(3,), isCuda=True)

# ====== convert SG to global envmap

weight_SG = intensity_SG
light_axis_world_SG = np.array([light_dir_abs_RAW[2], -light_dir_abs_RAW[0], light_dir_abs_RAW[1]])
light_axis_world_SG = light_axis_world_SG / np.linalg.norm(light_axis_world_SG)
cos_theta = light_axis_world_SG[2]
theta_SG = np.arccos(cos_theta) # [0, pi]
cos_phi = light_axis_world_SG[0] / np.sin(theta_SG)
sin_phi = light_axis_world_SG[1] / np.sin(theta_SG)
phi_SG = np.arctan2(sin_phi, cos_phi)
assert phi_SG >= -np.pi and phi_SG <= np.pi
ambient_SG = np.zeros(3, dtype=np.float32)

recImg = envOptim.renderSG(torch.tensor(theta_SG).reshape((1, 1, 1, 1, 1, 1)).cuda(), torch.tensor(phi_SG).reshape((1, 1, 1, 1, 1, 1)).cuda(), \
                           torch.tensor(lamb_SG).reshape((1, 1, 1, 1, 1)).cuda(), torch.tensor(weight_SG).reshape((1, 1, 3, 1, 1)).cuda(), \
                           ambient=torch.tensor(ambient_SG).reshape((1, 1, 3, 1, 1)).cuda())
im_envmap_gen = recImg.cpu().numpy().squeeze().transpose(1, 2, 0)
im_envmap_gen_uint8, _ = to_nonhdr(im_envmap_gen, scale=im_envmap_ori_scale)

%matplotlib widget

plt.figure(figsize=(8, 4))
ax = plt.subplot(121)
plt.imshow(im_envmap_ori_uint8)
ax.set_title('Ori env map (global RAW coords)')

ax = plt.subplot(122)
plt.imshow(im_envmap_gen_uint8)
ax.set_title('render SG env map (global RAW coords)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [365]:
plt.figure(figsize=(8, 4))
plt.imshow(im_envmap_ori_uint8)
ax.set_title('Ori env map (global RAW coords)')
plt.show()

plt.figure(figsize=(8, 4))
plt.imshow(im_envmap_gen_uint8)
ax.set_title('render SG env map - %s (global RAW coords)'%split_type)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## render original_img with GT scene

In [17]:
from utils_OR.DatasetCreation import SGOptim
import torch
from pathlib import Path
from utils_OR.utils_OR_imageops import loadHdr_simple, to_nonhdr


RAW_path = '/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/DatasetCreation'
xml_path = '/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes'
root_uv_mapped = Path('/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped')
envmap_root = '/newfoundland2/ruizhu/siggraphasia20dataset/EnvDataset/'

main_xml_file = Path(xml_path) / sequence['sub_name'] / sequence['scene_name'] / ('%s.xml'%sequence['meta_name'])

from copy import deepcopy
import cv2
from utils_OR.utils_OR_xml import gen_random_str, transformToXml
from utils_OR.utils_OR_imageops import loadHdr, loadImage, scaleHdr, load_OR_hdr_uint8

dataset_main_path = Path('/newfoundland2/ruizhu/siggraphasia20dataset')
scene_path = Path(xml_path) / sequence['sub_name'] / sequence['scene_name']

random_str = gen_random_str(5)
tmp_folder = Path('/home/ruizhu/Downloads/tmp/%s'%random_str)
tmp_folder.mkdir()
# rec_envmap_filename = tmp_folder / 'rec_envmap.hdr'
# cv2.imwrite(str(rec_envmap_filename), recImg_gen[:, :, ::-1].astype(np.float32))
# print('===== Written envmap to ' + str(rec_envmap_filename))

root = get_XML_root(main_xml_file)
rec_root = deepcopy(root )
emitters_env = rec_root.findall('emitter')
assert len(emitters_env) == 1
emitter_env = emitters_env[0]
assert emitter_env.get('type') == 'envmap'
# emitter_env.findall('string')[0].set('value', envmap_path)
# emitter_env.findall('string')[0].set('value', str(im_envmap_gen_path)) # use new SG env map!!!!!
# emitter_env.findall('float')[0].set('value', '0.0') # TURN OFF env map
rec_root.findall('sensor')[0].findall('sampler')[0].findall('integer')[0].set('value', '200') # Change samples
new_xml_path = str(main_xml_file).replace('.xml', '_rerennderGT_%s.xml'%random_str)

xmlString = transformToXml(rec_root )
with open(str(new_xml_path), 'w') as xmlOut:
    xmlOut.write(xmlString )
print('===== Written envmap XML to' + str(new_xml_path))

cam_idx = sequence_emitters['withinsequence_id'] - 1
cam_file = scene_path / 'cam.txt'
cam_file = Path(str(cam_file).replace('DiffMat', ''))
with open(cam_file, 'r') as fIn:
    lines = [x.strip() for x in fIn.readlines()]
lines_new = [str(1)] + lines[cam_idx*3+1:cam_idx*3+3+1]
new_cam_file = str(cam_file).replace('.txt', '_rerennderGT_%s.txt'%random_str)

with open(str(new_cam_file), 'w') as fIn:
    for line in lines_new:
        fIn.writelines(line)
        fIn.writelines('\n')
print('===== Written cam.txt to' + str(new_cam_file))

print('===== Rendering to ' + str(tmp_folder))
rendering_command = 'cd %s && optixRenderer -f %s -o %s -c %s -m 0 --gpuIds 0' \
    % (str(dataset_main_path), str(Path(new_xml_path).relative_to(dataset_main_path)), str(tmp_folder / 'rerender.hdr'), str(Path(new_cam_file).relative_to(scene_path)))
print(rendering_command)
os.system(rendering_command)
os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
os.system('mv %s %s/'%(new_cam_file, str(tmp_folder)))

im_rerender = loadHdr_simple(str(tmp_folder / 'rerender_1.rgbe'))
im_rerender_uint8, im_rerender_scale = to_nonhdr(im_rerender)
plt.figure(figsize=(8, 4))

plt.imshow(im_rerender_uint8)
plt.show()

===== Written envmap XML to/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml/scene0403_01/mainDiffMat_rerennderGT_93DI5.xml
===== Written cam.txt to/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml/scene0403_01/cam_rerennderGT_93DI5.txt
===== Rendering to /home/ruizhu/Downloads/tmp/93DI5
cd /newfoundland2/ruizhu/siggraphasia20dataset && optixRenderer -f code/Routine/scenes/xml/scene0403_01/mainDiffMat_rerennderGT_93DI5.xml -o /home/ruizhu/Downloads/tmp/93DI5/rerender.hdr -c cam_rerennderGT_93DI5.txt -m 0 --gpuIds 0


NameError: name 'loadHdr_simple' is not defined

## render original_envmap

In [302]:
main_xml_file

PosixPath('/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml1/scene0552_00/main.xml')

In [18]:
import xml.etree.ElementTree as et
from utils_OR.utils_OR_imageops import loadHdr_simple, to_nonhdr

random_str = gen_random_str(5)
tmp_folder = Path('/home/ruizhu/Downloads/tmp/%s'%random_str)
tmp_folder.mkdir()
# rec_envmap_filename = tmp_folder / 'rec_envmap.hdr'
# cv2.imwrite(str(rec_envmap_filename), recImg_gen[:, :, ::-1].astype(np.float32))
# print('===== Written envmap to ' + str(rec_envmap_filename))

root = get_XML_root(main_xml_file)
rec_root = deepcopy(root )
emitters_env = rec_root.findall('emitter')
assert len(emitters_env) == 1
emitter_env = emitters_env[0]
assert emitter_env.get('type') == 'envmap'
# emitter_env.findall('string')[0].set('value', str(rec_envmap_filename)) # use new SG env map!!!!!
# emitter_env.findall('float')[0].set('value', '0.0') # TURN OFF env map
rec_root.findall('sensor')[0].findall('sampler')[0].findall('integer')[0].set('value', '200') # Change samples

sensor = rec_root.findall('sensor')[0]
sensor.set('type', 'envmap')
film = sensor.findall('film')[0]
integers = film.findall('integer')
for integer in integers:
    if integer.get('name') == 'width':
        integer.set('value', '%d' % 512 )
    elif integer.get('name') == 'height':
        integer.set('value', '%d' % 256 )

center = [0., 0., 0.]
zAxis = [0, 1., 0]
camTransform = et.SubElement(sensor, 'transform')
camTransform.set('type', 'toWorld')
lookAt = et.SubElement(camTransform, 'lookAt')
lookAt.set('origin', '%.3f %.3f %.3f' \
        % (center[0], center[1], center[2] ) )
lookAt.set('target', '%.3f %.3f %.3f' \
        % (center[0] + zAxis[0], center[1] + zAxis[1], center[2] + zAxis[2] ) )
lookAt.set('up', '-1 0 0')

# shapes = [x for x in rec_root.findall('shape')]
# for x in shapes:
#     rec_root.remove(x)

# bsdfs = [x for x in rec_root.findall('bsdf')]
# for x in bsdfs:
#     rec_root.remove(x)

xmlString = transformToXml(rec_root )
new_xml_path = str(main_xml_file).replace('.xml', '_rerennder-GT-envmap_%s.xml'%random_str)
with open(str(new_xml_path), 'w') as xmlOut:
    xmlOut.write(xmlString )
print('===== Written envmap XML to' + str(new_xml_path))

print('===== Rendering to ' + str(tmp_folder))
rendering_command = 'cd %s && optixRenderer -f %s -o %s -m 0 --gpuIds 0' \
    % (str(dataset_main_path), str(Path(new_xml_path).relative_to(dataset_main_path)), str(tmp_folder / 'rerender-envmap.rgbe'),)
print(rendering_command)
os.system(rendering_command)
os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
os.system('mv %s %s/'%(new_cam_file, str(tmp_folder)))

im_rerender = loadHdr_simple(str(tmp_folder / 'rerender-envmap_1.rgbe'))
im_rerender_uint8, im_rerender_scale_ori = to_nonhdr(im_rerender)
plt.figure(figsize=(8, 4))

plt.imshow(im_rerender_uint8)
plt.show()

===== Written envmap XML to/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml/scene0403_01/mainDiffMat_rerennder-GT-envmap_453TW.xml
===== Rendering to /home/ruizhu/Downloads/tmp/453TW
cd /newfoundland2/ruizhu/siggraphasia20dataset && optixRenderer -f code/Routine/scenes/xml/scene0403_01/mainDiffMat_rerennder-GT-envmap_453TW.xml -o /home/ruizhu/Downloads/tmp/453TW/rerender-envmap.rgbe -m 0 --gpuIds 0
/home/ruizhu/Downloads/tmp/453TW/rerender-envmap_1.rgbe 340.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## convert est window to envmap

In [217]:
if_use_pred_layout = False
skip_uv_mapped = True
skip_ceiling = True
if_empty_room = True
layout_scale = 1.

In [223]:
import xml.etree.ElementTree as et


from pathlib import Path

import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding/utils_OR')
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding')

from utils_OR.utils_OR_xml import gen_random_str
from utils_OR.utils_OR_mesh import writeMesh_rect
from utils_OR.DatasetCreation.writeShapeToXml import addShape

random_str = gen_random_str(5)
tmp_folder = Path('/home/ruizhu/Downloads/tmp/%s'%random_str)
tmp_folder.mkdir()

root = get_XML_root(main_xml_file)
rec_root = deepcopy(root )

if if_empty_room:
    shapes = [x for x in rec_root.findall('shape')]
    for x in shapes:
        rec_root.remove(x)

    bsdfs = [x for x in rec_root.findall('bsdf')]
    for x in bsdfs:
        rec_root.remove(x)
        
if skip_uv_mapped:
    layout_shapes = [x for x in rec_root.findall('shape') if 'layoutMesh' in x.findall('string')[0].get('value')]
    for x in layout_shapes:
        rec_root.remove(x)
    

transform_R, transform_t = transform_to_total3d_coords_dict['transform_R'], transform_to_total3d_coords_dict['transform_t']

if if_use_pred_layout:
    layout = pred_layout_reindexed
    cell_info_grid = cell_info_grid_GT
    cam_R = cam_R_pred
else:
    layout = gt_layout
    cell_info_grid = cell_info_grid_PRED
    cam_R = cam_R_gt
    
faces_basis_indexes = [['x', 'z', '-y'], ['z', 'x', 'y'], ['y', 'z', 'x'], ['x', '-y', '-z'], ['z', 'y', '-x'], ['x', 'y', 'z']]

layout_basis_dict = {'x': layout['basis'][0], 'y': layout['basis'][1], 'z': layout['basis'][2], '-x': -layout['basis'][0], '-y': -layout['basis'][1], '-z': -layout['basis'][2]}
layout_coeffs_dict = {'x': layout['coeffs'][0], 'y': layout['coeffs'][1], 'z': layout['coeffs'][2], '-x': layout['coeffs'][0], '-y': layout['coeffs'][1], '-z': layout['coeffs'][2]}

for face_idx in range(6):
    valid_cell_list = []
    light_ratio_sum = 0.
    
    axis_1 = faces_basis_indexes[face_idx][0]
    axis_2 = faces_basis_indexes[face_idx][1]
    axis_3 = faces_basis_indexes[face_idx][2]
    basis_1_unit = layout_basis_dict[axis_1]
    basis_2_unit = layout_basis_dict[axis_2]
    basis_3_unit = layout_basis_dict[axis_3]
    origin_0 = layout['centroid'] - basis_1_unit * layout_coeffs_dict[axis_1] - basis_2_unit * layout_coeffs_dict[axis_2] - basis_3_unit * layout_coeffs_dict[axis_3]
    basis_1 = basis_1_unit * layout_coeffs_dict[axis_1] * 2 / grid_size
    basis_2 = basis_2_unit * layout_coeffs_dict[axis_2] * 2 / grid_size
    normal_outside = -basis_3_unit
    
    for cell_info in cell_info_grid:
        wall_idx, i, j = cell_info['wallidx_i_j']
        if wall_idx != face_idx:
            continue
#         print(wall_idx, i, j)

        if not(cell_info['obj_type'] == 'window' and cell_info['light_ratio'] > 0.1):
#             print(cell_info)
            continue
        
        x_ij = basis_1 * i + basis_2 * j + origin_0
        x_i1j = basis_1 * (i+1) + basis_2 * j + origin_0
        x_i1j1 = basis_1 * (i+1) + basis_2 * (j+1) + origin_0
        x_ij1 = basis_1 * i + basis_2 * (j+1) + origin_0
        verts = [[list(x_ij), list(x_i1j), list(x_i1j1), list(x_ij1)]]
        verts_array = np.array(verts).squeeze()
        verts_array = transform_R.T @ (verts_array.T - transform_t) # transform back to the RAW world
        verts_array = verts_array.T
        
        light_ratio_sum += cell_info['light_ratio']
        
        valid_cell_list.append({'basis_1_unit': basis_1_unit, 'basis_2_unit': basis_2_unit, 'light_ratio': cell_info['light_ratio'], 'center': np.mean(verts_array, axis=0), \
                                'center_axis1': (verts_array[1]+verts_array[0])/2., 'center_axis2': (verts_array[3]+verts_array[0])/2., 'i': i+0.5, 'j': j+0.5})
        
    if light_ratio_sum < 1.:
        verts_face_total3d = np.stack([origin_0, origin_0+basis_1*grid_size, origin_0+basis_1*grid_size+basis_2*grid_size, origin_0+basis_2*grid_size]).reshape((4,3))
#         verts_face_total3d = (transform_R @ verts_face.T + transform_t).T
#         verts_face_total3dcam = (cam_R.T @ verts_face_total3d.T).T # to zq cam coords
#         verts_face_zqworld = np.hstack([-verts_face_total3dcam[:, 2:3], verts_face_total3dcam[:, 1:2], verts_face_total3dcam[:, 0:1]])
#         verts_face = verts_face_zqworld
        verts_face = transform_R.T @ (verts_face_total3d.T - transform_t) # transform back to the RAW world
        verts_face = verts_face.T

        cellObjName = tmp_folder / ('rerendererGT_layout_face%d.obj'%face_idx)
        writeMesh_rect(str(cellObjName), verts_face)
        rec_root = addShape(rec_root, 'est_wall_%d'%face_idx, str(cellObjName), None, scaleValue=layout_scale)
        
    else:
        # merge window cells and write one wall as patches
        axis_1_cell = basis_1
        axis_2_cell = basis_2
        weights = np.array([x['light_ratio'] for x in valid_cell_list])
        weights = weights / np.sum(weights)
        weighted_center = np.sum(weights.reshape(-1, 1) * np.array([x['center'] for x in valid_cell_list]), 0)

        area_ori = np.linalg.norm(axis_1_cell) * np.linalg.norm(axis_2_cell)
        area_sum = sum([x['light_ratio'] for x in valid_cell_list]) * area_ori
        edge_length = np.sqrt(area_sum)

        coeffs_axis1 = (np.array([x['i'] for x in valid_cell_list])).reshape(-1, 1)
        centers_axis1 = np.outer(coeffs_axis1, basis_1.reshape(1, 3))
        weighted_center_axis1 = np.sum(weights.reshape(-1, 1) * centers_axis1, 0) # offset w.r.t. origin_0
        coeffs_axis2 = (np.array([x['j'] for x in valid_cell_list])).reshape(-1, 1)
        centers_axis2 = np.outer(coeffs_axis2, basis_2.reshape(1, 3))
        weighted_center_axis2 = np.sum(weights.reshape(-1, 1) * centers_axis2, 0) # offset w.r.t. origin_0

        weighted_center_axis1_coeff = np.sum(weights.reshape(-1, 1) * coeffs_axis1, 0).item()
        weighted_center_axis2_coeff = np.sum(weights.reshape(-1, 1) * coeffs_axis2, 0).item()
        half_edge_axis1_coeff = edge_length / 2. / np.linalg.norm(basis_1)
        half_edge_axis2_coeff = edge_length / 2. / np.linalg.norm(basis_2)
        c1, c2 = weighted_center_axis1_coeff, weighted_center_axis2_coeff
        h1, h2 = half_edge_axis1_coeff, half_edge_axis2_coeff

        for patch_idx, coeffs in enumerate([\
                                            [[0., 0.], [c1-h1, 0], [c1-h1, c2-h2], [0., c2-h2]], \
                                            [[c1-h1, 0], [c1+h1, 0], [c1+h1, c2-h2], [c1-h1, c2-h2]], \
                                            [[c1+h1, 0], [grid_size, 0], [grid_size, c2-h2], [c1+h1, c2-h2]], \
                                            [[0, c2-h2], [c1-h1, c2-h2], [c1-h1, c2+h2], [0, c2+h2]], \
                                            [[c1+h1, c2-h2], [grid_size, c2-h2], [grid_size, c2+h2], [c2+h1, c2+h2]], \
                                            [[0, c2+h2], [c1+h1, c2+h2], [c1+h1, grid_size], [0, grid_size]], \
                                            [[c1-h1, c2+h2], [c1+h1, c2+h2], [c1+h1, grid_size], [c1-h1, grid_size]], \
                                            [[c1+h1, c2+h2], [grid_size, c2+h2], [grid_size, grid_size], [c1+h1, grid_size]]
                                             ]):
            patch_total3d = np.stack([origin_0 + basis_1*coeff_1 + basis_2*coeff_2 for coeff_1, coeff_2 in coeffs])
            patch = transform_R.T @ (patch_total3d.T - transform_t) # transform back to the RAW world
            patch = patch.T

#             patch_total3d = (transform_R @ patch.T + sequence['transform_t']).T
#             patch_total3dcam = (cam_R_pred.T @ patch_total3d.T).T # to zq cam coords
#             patch_zqworld = np.hstack([-patch_total3dcam[:, 2:3], patch_total3dcam[:, 1:2], patch_total3dcam[:, 0:1]])
#             patch = patch_zqworld

            cellObjName = tmp_folder / ('rerendererGT_window_on%d_%d.obj'%(face_idx, patch_idx))
#             writeMesh_rect(str(cellObjName), patch)
#             rec_root = addShape(rec_root, 'est_window2_on%d_%d'%(face_idx, patch_idx), str(cellObjName), None, scaleValue=layout_scale)

{'obj_type': 'window', 'emitter_info': {'light_dir_offset': array([-0.1817,  0.6135, -0.3107], dtype=float32), 'emitter_outdirs_meshgrid_Total3D_outside_abs': array([[[ 0.7147, -0.0191,  0.6991],
        [ 0.7035, -0.0545,  0.7086],
        [ 0.6829, -0.0815,  0.726 ],
        [ 0.6558, -0.0961,  0.7488],
        [ 0.6266, -0.0961,  0.7734],
        [ 0.5996, -0.0815,  0.7962],
        [ 0.5789, -0.0545,  0.8136],
        [ 0.5677, -0.0191,  0.823 ],
        [ 0.5677,  0.0191,  0.823 ],
        [ 0.5789,  0.0545,  0.8136],
        [ 0.5996,  0.0815,  0.7962],
        [ 0.6266,  0.0961,  0.7734],
        [ 0.6558,  0.0961,  0.7488],
        [ 0.6829,  0.0815,  0.726 ],
        [ 0.7035,  0.0545,  0.7086],
        [ 0.7147,  0.0191,  0.6991]],

       [[ 0.8343, -0.0566,  0.5484],
        [ 0.8012, -0.1613,  0.5763],
        [ 0.7399, -0.2414,  0.6279],
        [ 0.6599, -0.2847,  0.6953],
        [ 0.5733, -0.2847,  0.7683],
        [ 0.4932, -0.2414,  0.8357],
        [ 0.432 , -0.1613

In [229]:
cell_info['emitter_info'].keys()

dict_keys(['light_dir_offset', 'emitter_outdirs_meshgrid_Total3D_outside_abs', 'normal_outside_Total3D_single', 'intensity_scalelog', 'intensity_scaled01', 'intensity', 'lamb'])

In [225]:
cell_info.keys()

dict_keys(['obj_type', 'emitter_info', 'wallidx_i_j', 'light_ratio', 'cell_center', 'verts', 'normal_outside'])

In [369]:
sensor = rec_root.findall('sensor')[0]
sensor.set('type', 'envmap')
film = sensor.findall('film')[0]
integers = film.findall('integer')
for integer in integers:
    if integer.get('name') == 'width':
        integer.set('value', '%d' % 512 )
    elif integer.get('name') == 'height':
        integer.set('value', '%d' % 256 )

center = [0., 0., 0.]
zAxis = [0, 1., 0]
camTransform = et.SubElement(sensor, 'transform')
camTransform.set('type', 'toWorld')
lookAt = et.SubElement(camTransform, 'lookAt')
lookAt.set('origin', '%.3f %.3f %.3f' \
        % (center[0], center[1], center[2] ) )
lookAt.set('target', '%.3f %.3f %.3f' \
        % (center[0] + zAxis[0], center[1] + zAxis[1], center[2] + zAxis[2] ) )
lookAt.set('up', '-1 0 0')

rec_root.findall('sensor')[0].findall('sampler')[0].findall('integer')[0].set('value', '200') # Change samples

new_xml_path = str(main_xml_file).replace('.xml', '_emptyRoom_%s.xml'%random_str)

xmlString = transformToXml(rec_root )
with open(str(new_xml_path), 'w') as xmlOut:
    xmlOut.write(xmlString )
print('===== Written envmap XML to' + str(new_xml_path))


print('===== Rendering to ' + str(tmp_folder))
rendering_command = 'cd %s && optixRenderer -f %s -o %s -m 0 --gpuIds 0' \
    % (str(dataset_main_path), str(Path(new_xml_path).relative_to(dataset_main_path)), str(tmp_folder / 'rerender-envmap.rgbe'),)
print(rendering_command)
os.system(rendering_command)
os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
os.system('mv %s %s/'%(new_cam_file, str(tmp_folder)))

im_rerender = loadHdr_simple(str(tmp_folder / 'rerender-envmap_1.rgbe'))
im_rerender_uint8, _ = to_nonhdr(im_rerender, scale=im_rerender_scale_ori)
plt.figure(figsize=(8, 4))
plt.imshow(im_rerender_uint8)
plt.show()

===== Written envmap XML to/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml1/scene0552_00/main_emptyRoom_HYL43.xml
===== Rendering to /home/ruizhu/Downloads/tmp/HYL43
cd /newfoundland2/ruizhu/siggraphasia20dataset && optixRenderer -f code/Routine/scenes/xml1/scene0552_00/main_emptyRoom_HYL43.xml -o /home/ruizhu/Downloads/tmp/HYL43/rerender-envmap.rgbe -m 0 --gpuIds 0
/home/ruizhu/Downloads/tmp/HYL43/rerender-envmap_1.rgbe 458.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [257]:
moved_path = tmp_folder / Path(new_xml_path).name

print(moved_path)

os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
# os.system('mv %s %s/'%(new_cam_file, str(tmp_folder)))

/home/ruizhu/Downloads/tmp/EAODG/main_emptyRoom_EAODG.xml


256

In [258]:
from utils_OR.utils_OR_xml import xml_scene_to_mesh
# new_xml_path_relocated = str(Path(tmp_folder) / Path(new_xml_path).name)
mesh_path = '../test_mesh_RECON_oneBoxLight_bunny.obj'
if Path(mesh_path).exists():
    Path(mesh_path).unlink()
insertion_path = '/home/ruizhu/Documents/Projects/Total3DUnderstanding/VirtualObjectInsertion'
xml_scene_to_mesh(str(moved_path), mesh_path, skip_uv_mapped=skip_uv_mapped, base_path=str(insertion_path))

print('===== Layout mesh written to: %s'%(str(mesh_path)))

13it [00:00, 3770.29it/s]

Mesh saved to ../test_mesh_RECON_oneBoxLight_bunny.obj; loaded from /home/ruizhu/Downloads/tmp/EAODG/main_emptyRoom_EAODG.xml.
===== Layout mesh written to: ../test_mesh_RECON_oneBoxLight_bunny.obj


## lamps

In [346]:
def get_corners_of_bb3d_no_index(basis, coeffs, centroid):
    corners = np.zeros((8, 3))
    coeffs = np.abs(coeffs)
    corners[0, :] = - basis[0, :] * coeffs[0] + basis[1, :] * coeffs[1] - basis[2, :] * coeffs[2]
    corners[1, :] = - basis[0, :] * coeffs[0] + basis[1, :] * coeffs[1] + basis[2, :] * coeffs[2]
    corners[2, :] = + basis[0, :] * coeffs[0] + basis[1, :] * coeffs[1] + basis[2, :] * coeffs[2]
    corners[3, :] = + basis[0, :] * coeffs[0] + basis[1, :] * coeffs[1] - basis[2, :] * coeffs[2]

    corners[4, :] = - basis[0, :] * coeffs[0] - basis[1, :] * coeffs[1] - basis[2, :] * coeffs[2]
    corners[5, :] = - basis[0, :] * coeffs[0] - basis[1, :] * coeffs[1] + basis[2, :] * coeffs[2]
    corners[6, :] = + basis[0, :] * coeffs[0] - basis[1, :] * coeffs[1] + basis[2, :] * coeffs[2]
    corners[7, :] = + basis[0, :] * coeffs[0] - basis[1, :] * coeffs[1] - basis[2, :] * coeffs[2]
    corners = corners + np.tile(centroid, (8, 1))
    return corners

In [347]:
if_use_pred_layout = False
skip_uv_mapped = False
skip_ceiling = True
if_empty_room = False
if_remove_emitter = True
layout_scale = 1.

In [356]:
random_str = gen_random_str(5)
tmp_folder = Path('/home/ruizhu/Downloads/tmp/%s'%random_str)
tmp_folder.mkdir()


root = get_XML_root(main_xml_file)
rec_root = deepcopy(root )

if if_empty_room:
    shapes = [x for x in rec_root.findall('shape')]
    for x in shapes:
        rec_root.remove(x)

    bsdfs = [x for x in rec_root.findall('bsdf')]
    for x in bsdfs:
        rec_root.remove(x)
        
if skip_uv_mapped:
    layout_shapes = [x for x in rec_root.findall('shape') if 'layoutMesh' in x.findall('string')[0].get('value')]
    for x in layout_shapes:
        rec_root.remove(x)
        
if if_remove_emitter:
    emitter_shapes = [x for x in rec_root.findall('shape') if len(x.findall('emitter'))>0]
    for x in emitter_shapes:
        rec_root.remove(x)    
        
sensor = rec_root.findall('sensor')[0]
sensor.set('type', 'envmap')
film = sensor.findall('film')[0]
integers = film.findall('integer')
for integer in integers:
    if integer.get('name') == 'width':
        integer.set('value', '%d' % 512 )
    elif integer.get('name') == 'height':
        integer.set('value', '%d' % 256 )

center = [0., 0., 0.]
zAxis = [0, 1., 0]
camTransform = et.SubElement(sensor, 'transform')
camTransform.set('type', 'toWorld')
lookAt = et.SubElement(camTransform, 'lookAt')
lookAt.set('origin', '%.3f %.3f %.3f' \
        % (center[0], center[1], center[2] ) )
lookAt.set('target', '%.3f %.3f %.3f' \
        % (center[0] + zAxis[0], center[1] + zAxis[1], center[2] + zAxis[2] ) )
lookAt.set('up', '-1 0 0')

rec_root.findall('sensor')[0].findall('sampler')[0].findall('integer')[0].set('value', '200') # Change samples

In [370]:
from utils_OR.DatasetCreation.writeShapeToXml import addAreaLight

cells_vis_info_list_filtered = [x for x in cell_info_grid if x['obj_type'] == 'obj']
valid_cell_list = []

for x_idx, cell_info in enumerate(cells_vis_info_list_filtered):
    if cell_info['light_ratio'] < 0.05:
        continue
    intensity = cell_info['emitter_info']['intensity']
    verts_RAW = transform_R.T @ ((np.array(cell_info['verts']).squeeze()).T - transform_t) # transform back to the RAW world
    verts_RAW = verts_RAW.T
    verts_RAW = (verts_RAW - np.mean(verts_RAW, axis=0, keepdims=True)) * cell_info['light_ratio'] + np.mean(verts_RAW, axis=0, keepdims=True)
    verts_RAW[:, 1] = verts_RAW[:, 1] - 0.1

    wall_idx, i, j = cell_info['wallidx_i_j']
    
    axis_1 = faces_basis_indexes[wall_idx][0]
    axis_2 = faces_basis_indexes[wall_idx][1]
    axis_3 = faces_basis_indexes[wall_idx][2]
    basis_1_unit = layout_basis_dict[axis_1]
    basis_2_unit = layout_basis_dict[axis_2]
    basis_3_unit = layout_basis_dict[axis_3]
    origin_0 = layout['centroid'] - basis_1_unit * layout_coeffs_dict[axis_1] - basis_2_unit * layout_coeffs_dict[axis_2] - basis_3_unit * layout_coeffs_dict[axis_3]
    basis_1 = basis_1_unit * layout_coeffs_dict[axis_1] * 2 / grid_size
    basis_2 = basis_2_unit * layout_coeffs_dict[axis_2] * 2 / grid_size
    basis_3 = basis_3_unit * layout_coeffs_dict[axis_3] * 2 / grid_size

    normal_outside = -basis_3_unit
    
#     valid_cell_list.append({'intensity': intensity, 'axis_1': basis_1, 'axis_2': basis_2, 'light_ratio': x['light_ratio'], 'center': np.mean(verts, axis=0)})


    edge_length = np.sqrt(cell_info['light_ratio'])
    box_center = basis_1 * (i+0.5) + basis_2 * (j+0.5) + basis_3 * 0.5 + origin_0
    
    lamp_box = get_corners_of_bb3d_no_index(np.vstack([basis_1_unit, basis_2_unit, basis_3_unit]), [edge_length/2., edge_length/2., edge_length/2.], box_center)
    lamp_box_RAW = transform_R.T @ (lamp_box.T - transform_t) # transform back to the RAW world
    lamp_box_RAW = lamp_box_RAW.T
    
#     print(verts_RAW.mean(axis=0))
#     print((transform_R.T @ (box_center.reshape(3, 1) - transform_t)).T)
#     print(lamp_box_RAW, lamp_box_RAW.mean(axis=0))
    
    cellObjName = tmp_folder / ('rerendererGT_lampCell_%d.obj'%x_idx)
#     writeMesh_rect(str(cellObjName), verts_RAW)
    writeMesh_rect(str(cellObjName), lamp_box_RAW)
    
#     print(verts_RAW)

    print(cell_info['light_ratio'])
    
#     intensity = np.ones(3)*10.
    rec_root = addAreaLight(rec_root, 'est_lamp_on%d_%d'%(wall_idx, x_idx), str(cellObjName), rgbColor=intensity)

0.08838832
0.24534026
0.17897984
0.1347794


In [220]:
cells_vis_info_list_filtered

NameError: name 'cells_vis_info_list_filtered' is not defined

In [371]:
new_xml_path = str(main_xml_file).replace('.xml', '_emptyRoomWithLamp_%s.xml'%random_str)

xmlString = transformToXml(rec_root )
with open(str(new_xml_path), 'w') as xmlOut:
    xmlOut.write(xmlString )
print('===== Written envmap XML to' + str(new_xml_path))


print('===== Rendering to ' + str(tmp_folder))
rendering_path = tmp_folder / 'rerender-envmap_1.rgbe'
if rendering_path.exists():
    rendering_path.unlink()
rendering_command = 'cd %s && optixRenderer -f %s -o %s -m 0 --gpuIds 0' \
    % (str(dataset_main_path), str(Path(new_xml_path).relative_to(dataset_main_path)), str(tmp_folder / 'rerender-envmap.rgbe'),)
print(rendering_command)
os.system(rendering_command)
os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
os.system('mv %s %s/'%(new_cam_file, str(tmp_folder)))

im_rerender = loadHdr_simple(str(tmp_folder / 'rerender-envmap_1.rgbe'))
im_rerender_uint8, _ = to_nonhdr(im_rerender, scale=im_rerender_scale_ori)

%matplotlib widget
plt.figure(figsize=(8, 4))
plt.imshow(im_rerender_uint8)
plt.show()
    

moved_path = tmp_folder / Path(new_xml_path).name

print(moved_path)

os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
# os.system('mv %s %s/'%(new_cam_file, str(tmp_folder)))

===== Written envmap XML to/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml1/scene0552_00/main_emptyRoomWithLamp_HYL43.xml
===== Rendering to /home/ruizhu/Downloads/tmp/HYL43
cd /newfoundland2/ruizhu/siggraphasia20dataset && optixRenderer -f code/Routine/scenes/xml1/scene0552_00/main_emptyRoomWithLamp_HYL43.xml -o /home/ruizhu/Downloads/tmp/HYL43/rerender-envmap.rgbe -m 0 --gpuIds 0
/home/ruizhu/Downloads/tmp/HYL43/rerender-envmap_1.rgbe 1280.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/ruizhu/Downloads/tmp/HYL43/main_emptyRoomWithLamp_HYL43.xml


256

In [353]:
from utils_OR.utils_OR_xml import xml_scene_to_mesh
# new_xml_path_relocated = str(Path(tmp_folder) / Path(new_xml_path).name)
mesh_path = '../test_mesh_RECON_oneBoxLight_bunny.obj'
if Path(mesh_path).exists():
    Path(mesh_path).unlink()
insertion_path = '/home/ruizhu/Documents/Projects/Total3DUnderstanding/VirtualObjectInsertion'
xml_scene_to_mesh(str(moved_path), mesh_path, skip_uv_mapped=skip_uv_mapped, base_path=str(insertion_path))

print('===== Scene mesh written to: %s'%(str(mesh_path)))

17it [00:00, 1544.83it/s]

/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_layout_face0.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_layout_face1.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_layout_face2.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_0.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_1.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_2.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_3.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_4.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_5.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_6.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_window_on3_7.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_layout_face4.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_layout_face5.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_lampCell_0.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_lampCell_1.obj
/home/ruizhu/Downloads/tmp/WZE9F/rerendererGT_lampCell_2.ob

In [317]:
# remove added lamps
est_lamp_shapes = [x for x in rec_root.findall('shape') if 'est_lamp' in x.findall('string')[0].get('value')]
for x in est_lamp_shapes:
    rec_root.remove(x)

## add meshes

In [440]:
from utils.utils_total3D.utils_OR_mesh import loadMesh, writeMesh

# remove added scene_objs_GT
scene_objs_shapes = [x for x in rec_root.findall('shape') if 'scene_obj' in x.get('id')]
for x in scene_objs_shapes:
    print(x)
    rec_root.remove(x)

mesh_objs = return_dict['mesh_objs']['GT']
for mesh_idx, mesh_obj_dict in enumerate(mesh_objs):
    mesh_path = tmp_folder / ('rerendererGT_obj_%d.obj'%mesh_idx)
    v_total3d = mesh_obj_dict['v']
    v_RAW = transform_R.T @ (v_total3d.T - transform_t) # transform back to the RAW world
    v_RAW = v_RAW.T
    writeMesh(str(mesh_path), v_RAW, mesh_obj_dict['f'])
    
    print(mesh_obj_dict['obj_path'])
    print(str(mesh_path))
    rec_root = addShape(rec_root, 'scene_obj_%d'%mesh_idx, str(mesh_path), None, scaleValue=1.)
#     rec_root = addShape(rec_root, 'scene_obj_%d'%mesh_idx, mesh_obj_dict['obj_path'], None, scaleValue=1.)

# scene_mesh_GT = return_dict['mesh_paths']['GT']
# scene_mesh_GT = Path('/home/ruizhu/Documents/Projects/semanticInverse/notebooks') / scene_mesh_GT
# rec_root = addShape(rec_root, 'scene_objs_GT', str(scene_mesh_GT), None, scaleValue=1.)

/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped/02933112/a3a6f9e80956ec6f4035f93ab9531db/alignedNew.obj
/home/ruizhu/Downloads/tmp/HYL43/rerendererGT_obj_0.obj
/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped/04379243/3b64bf8731a6a2a63c5e1addd9922bf2/alignedNew.obj
/home/ruizhu/Downloads/tmp/HYL43/rerendererGT_obj_1.obj
/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped/03001627/be38dc611c4bdef7f1cdfc0a8f38f2e/alignedNew.obj
/home/ruizhu/Downloads/tmp/HYL43/rerendererGT_obj_2.obj
/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped/03001627/be38dc611c4bdef7f1cdfc0a8f38f2e/alignedNew.obj
/home/ruizhu/Downloads/tmp/HYL43/rerendererGT_obj_3.obj
/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped/04379243/899f815eaad62b56d8cad143689f8b51/alignedNew.obj
/home/ruizhu/Downloads/tmp/HYL43/rerendererGT_obj_4.obj
/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped/02747177/8a50955facbac20ecf19858fd1963d10/alignedNew.obj
/home/ruizhu/Downloads/tmp/HYL43/rerendererGT_obj_5.obj
/

In [441]:
new_xml_path = str(main_xml_file).replace('.xml', '_emptyRoomWithLampObjs_%s.xml'%random_str)

xmlString = transformToXml(rec_root )
with open(str(new_xml_path), 'w') as xmlOut:
    xmlOut.write(xmlString )
print('===== Written envmap XML to' + str(new_xml_path))

print('===== Rendering to ' + str(tmp_folder))
rendering_path = tmp_folder / 'rerender-envmap_1.rgbe'
if rendering_path.exists():
    rendering_path.unlink()
rendering_command = 'cd %s && optixRenderer -f %s -o %s -m 0 --gpuIds 0' \
    % (str(dataset_main_path), str(Path(new_xml_path).relative_to(dataset_main_path)), str(tmp_folder / 'rerender-envmap.rgbe'),)
print(rendering_command)
os.system(rendering_command)

# os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))

im_rerender = loadHdr_simple(str(tmp_folder / 'rerender-envmap_1.rgbe'))
im_rerender_uint8, _ = to_nonhdr(im_rerender, scale=im_rerender_scale_ori)

%matplotlib widget
plt.figure(figsize=(8, 4))
plt.imshow(im_rerender_uint8)
plt.show()
    

moved_path = tmp_folder / Path(new_xml_path).name

print(moved_path)

os.system('mv %s %s/'%(new_xml_path, str(tmp_folder)))
# os.system('mv %s %s/'%(new_cam_file, str(tmp_folder)))

===== Written envmap XML to/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml1/scene0552_00/main_emptyRoomWithLampObjs_HYL43.xml
===== Rendering to /home/ruizhu/Downloads/tmp/HYL43
cd /newfoundland2/ruizhu/siggraphasia20dataset && optixRenderer -f code/Routine/scenes/xml1/scene0552_00/main_emptyRoomWithLampObjs_HYL43.xml -o /home/ruizhu/Downloads/tmp/HYL43/rerender-envmap.rgbe -m 0 --gpuIds 0
/home/ruizhu/Downloads/tmp/HYL43/rerender-envmap_1.rgbe 1032.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/ruizhu/Downloads/tmp/HYL43/main_emptyRoomWithLampObjs_HYL43.xml


0

# utils

In [99]:
import os.path as osp
import struct
from PIL import Image

def loadBinary(imName, channels = 1, dtype=np.float32, if_resize=True):
    assert dtype in [np.float32, np.int32], 'Invalid binary type outside (np.float32, np.int32)!'
    if not(osp.isfile(imName ) ):
        assert(False ), '%s doesnt exist!'%imName
    with open(imName, 'rb') as fIn:
        hBuffer = fIn.read(4)
        height = struct.unpack('i', hBuffer)[0]
        wBuffer = fIn.read(4)
        width = struct.unpack('i', wBuffer)[0]
        dBuffer = fIn.read(4 * channels * width * height )
        if dtype == np.float32:
            decode_char = 'f'
        elif dtype == np.int32:
            decode_char = 'i'
        depth = np.asarray(struct.unpack(decode_char * channels * height * width, dBuffer), dtype=dtype)
        depth = depth.reshape([height, width, channels] )

        depth = np.squeeze(depth)

    return depth[np.newaxis, :, :]

def loadImage(imName, isGama = False):
    if not(osp.isfile(imName ) ):
        assert(False), 'File does not exist: ' + imName 

    im = Image.open(imName)

    im = np.asarray(im, dtype=np.float32)
    if isGama:
        im = (im / 255.0) ** 2.2
        im = 2 * im - 1
    else:
        im = (im - 127.5) / 127.5
    if len(im.shape) == 2:
        im = im[:, np.newaxis]
    im = np.transpose(im, [2, 0, 1] )

    return im

